In [1]:
import pandas as pd
import altair as alt

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [3]:
vr = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/vehicles/veh1153a.csv')

In [4]:
chart_df = vr[vr['Geography'] == 'United Kingdom']
chart_df = chart_df[chart_df['Date Interval'] == 'Annual']
chart_df = chart_df[chart_df['BodyType'] == 'Cars']
chart_df = chart_df[chart_df['Units'] == 'Thousands']
chart_df = chart_df[chart_df['Keepership'] == 'Total']

In [5]:
chart_df = chart_df.drop(['Geography', 'Date Interval', 'Units', 'BodyType', 'Keepership'], axis=1)

In [6]:
chart_df['Hybrid electric'] = chart_df['Hybrid electric (petrol)'] + chart_df['Hybrid electric (diesel)']
chart_df['Plug-in'] = chart_df['Plug-in hybrid electric (petrol)'] + chart_df['Plug-in hybrid electric (diesel)']

In [7]:
chart_df = chart_df.melt(id_vars=['Date'], value_vars=['Petrol', 'Diesel', 'Hybrid electric (petrol)', 'Hybrid electric (diesel)', 'Plug-in hybrid electric (petrol)','Plug-in hybrid electric (diesel)', 'Battery electric', 'Range extended electric', 'Fuel cell electric', 'Gas', 'Others', 'Total', 'Plugin', 'ZEV' ,'Hybrid electric', 'Plug-in'], value_name='Registrations', var_name='Type')

In [8]:
chart_df = chart_df[chart_df['Type'].isin(['Petrol', 'Diesel', 'Hybrid electric', 'ZEV', 'Plug-in'])]

In [9]:
# 2. Find last year for each Type
last_points = chart_df.loc[chart_df.groupby('Type')['Date'].idxmax()]

# 3. Build the line chart
line = alt.Chart(chart_df).mark_line().encode(
    x=alt.X('Date:N', axis=alt.Axis(title=''), scale=alt.Scale(padding=0)),
    y=alt.Y('Registrations', axis=alt.Axis(format=',.0f', 
                                           title='Registrations', 
                                           titleX=-80, 
                                           titleFontSize=14, 
                                           labelExpr='datum.label + "K"')),
    color=alt.Color('Type', legend=None)
)

# 4. Add text labels at the end
text = alt.Chart(last_points).mark_text(
    align='left',
    dx=5,
    fontWeight='bold',
    fontSize=13
).encode(
    x=alt.X('Date:N'),
    y=alt.Y('Registrations'),
    text='Type',
    color=alt.Color('Type', legend=None)
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: Vehicle licensing statistics data tables - DoT and DVLA"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=10,
    fontStyle='italic',
    dy=175,
    x=-15
).encode(
    text='text'
)

# 5. Combine and display
chart = (line + text + footnote).properties(title= 'First-time Car Registrations by Fuel Type', width=450).configure_title(
    fontSize=20  # Increase title font size here
)
chart

alt.LayerChart(...)

In [10]:
# Save to png
chart.save('vehicle_registrations.png', scale_factor=2)
# Save to json
chart.save('vehicle_registrations.json', scale_factor=2)